# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/model/')

In [2]:
import pandas as pd
import numpy as np
import torch
import pickle
import itertools
import random
from torch.utils.tensorboard import SummaryWriter
import os

from utils import *

In [3]:
from pipeline_config import *

# Seeds for Comparisons:

In [5]:
train_dataset_dict = pd.read_pickle(train_dataset_dict_path)
train_dataset_dict['Air transportation']

,Inflation t-12,Inflation t-11,Inflation t-10,Inflation t-9,Inflation t-8,Inflation t-7,Inflation t-6,Inflation t-5,Inflation t-4,Inflation t-3,Inflation t-2,Inflation t-1,Inflation t,Inflation t+1
20875,0.570731,0.327601,-0.898332,-2.726713,0.316957,-2.042490,-0.542847,0.387447,1.564977,3.531667,2.565304,-0.418236,-0.166806,0.890334
20876,0.890334,0.570731,0.327601,-0.898332,-2.726713,0.316957,-2.042490,-0.542847,0.387447,1.564977,3.531667,2.565304,-0.418236,-0.242033
20877,-0.242033,0.890334,0.570731,0.327601,-0.898332,-2.726713,0.316957,-2.042490,-0.542847,0.387447,1.564977,3.531667,2.565304,1.124510
20878,1.124510,-0.242033,0.890334,0.570731,0.327601,-0.898332,-2.726713,0.316957,-2.042490,-0.542847,0.387447,1.564977,3.531667,0.478089
20879,0.478089,1.124510,-0.242033,0.890334,0.570731,0.327601,-0.898332,-2.726713,0.316957,-2.042490,-0.542847,0.387447,1.564977,0.633917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20953,-5.287319,-3.667078,4.234616,0.742648,3.382247,-2.980063,1.508149,-14.904457,20.988839,-1.540863,1.851905,-21.752029,3.157085,-7.622146
20954,-7.622146,-5.287319,-3.667078,4.234616,0.742648,3.382247,-2.980063,1.508149,-14.904457,20.988839,-1.540863,1.851905,-21.752029,-0.387097
20955,-0.387097,-7.622146,-5.287319,-3.667078,4.234616,0.742648,3.382247,-2.980063,1.508149,-14.904457,20.988839,-1.540863,1.851905,2.173999
20956,2.173999,-0.387097,-7.622146,-5.287319,-3.667078,4.234616,0.742648,3.382247,-2.980063,1.508149,-14.904457,20.988839,-1.540863,7.139353


In [4]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Data

In [5]:
#with open(son_parent_path, 'rb') as f:
#    son_parent_dict = pickle.load(f)
son_parent_dict = pd.read_pickle(son_parent_path)

#with open(train_dataset_dict_path, 'rb') as f:
#    train_dataset_dict = pickle.load(f)
train_dataset_dict = pd.read_pickle(train_dataset_dict_path)

#with open(test_dataset_dict_path, 'rb') as f:
#    test_dataset_dict = pickle.load(f)
test_dataset_dict = pd.read_pickle(test_dataset_dict_path)

#with open(coefficient_dict_path, 'rb') as f:
#    coefficient_dict = pickle.load(f)
coefficient_dict = pd.read_pickle(coefficient_dict_path)

#with open(category_id_to_category_name_path, 'rb') as f:
#    category_id_to_name_dict = pickle.load(f)
category_id_to_name_dict = pd.read_pickle(category_id_to_category_name_path)
    
#with open(categories_per_indent_path, 'rb') as f:
#    categories_per_indent_dict = pickle.load(f)
categories_per_indent_dict = pd.read_pickle(categories_per_indent_path)

#with open(reversed_parent_to_son_list_path, 'rb') as f:
#    parent_to_son_list_dict = pickle.load(f)
parent_to_son_list_dict = pd.read_pickle(reversed_parent_to_son_list_path)

#with open(hgru_weight_path, 'rb') as f:
#    hgru_weight_dict = pickle.load(f)
hgru_weight_dict = pd.read_pickle(hgru_weight_path)

weights_path = weightspath

In [6]:
sorted(list(categories_per_indent_dict.keys()),reverse=True)

[6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.0]

In [7]:
for id in category_id_to_name_dict.keys():
    if category_id_to_name_dict[id] == 'New cars':
        print(id)

In [8]:
for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
    for category in categories_per_indent_dict[indent]:
        if category == 5782.0:
            print(indent)

In [9]:
5782.0 in categories_per_indent_dict[5]

False

In [10]:
import os
from pathlib import Path
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

# Hierarchical GRU

In [11]:
def bi_directional_model(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)

    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    Lr = 0.0961278669993956
    loss_coef_3 = 1.3253456681466107*np.exp(-10)

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
    #for indent in [5]:
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0
            else:
                loss_coef_1 = 3.1714644592370186*np.exp(-10) #post optuna

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            else:
                loss_coef_2 = 6.8094717681531*np.exp(-7) #post optuna
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, path = saving_param_path)

In [12]:
bi_directional_models = bi_directional_model(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, hgru_weight_dict, coefficient_dict)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/models_weights/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/models_weights/
num categories: 1
category id|name: 345|All other miscellaneous food preparations
------------------------------------------------------------------
num categories: 2
category id|name: 316|Bacon
------------------------------------------------------------------
num categories: 3
category id|name: 332|Canned fruit
------------------------------------------------------------------
num categories: 4
category id|name: 341|Canned infant or junior foods
------------------------------------------------------------------
num categories: 5
category id|name: 318|Canned salmon
------------------------------------------------------------------
num catego

In [13]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        if category == 'Food and beverages':
            continue
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model =GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
        basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=0.00005)
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [14]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [15]:
categories

['Air transportation',
 'Audio equipment',
 'Bakery products',
 'Beer purchased from stores',
 'Beer served in licensed establishments',
 'Books and reading material (excluding textbooks',
 'Cereal products (excluding baby food',
 'Child care services',
 'City bus and subway transportation',
 'Coffee and tea',
 'Condiments, spices and vinegars',
 'Dairy products',
 'Dental care services',
 'Detergents and soaps (other than personal care',
 'Digital computing equipment and devices',
 'Edible fats and oils',
 'Eggs',
 'Eye care goods',
 'Eye care services',
 'Financial services',
 'Fish',
 'Fresh fruit',
 'Fresh or frozen meat (excluding poultry',
 'Fresh or frozen poultry',
 'Fresh vegetables',
 'Furniture',
 'Gasoline',
 'Household appliances',
 'Household textiles',
 'Housekeeping services',
 'Internet access services',
 'Liquor purchased from stores',
 'Liquor served in licensed establishments',
 'Magazines and periodicals',
 'Medicinal and pharmaceutical products',
 'Newspapers',
 '

In [16]:
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

Air transportation
Audio equipment
Bakery products
Beer purchased from stores
Beer served in licensed establishments
Books and reading material (excluding textbooks
Cereal products (excluding baby food
Child care services
City bus and subway transportation
Coffee and tea
Condiments, spices and vinegars
Dairy products
Dental care services
Detergents and soaps (other than personal care
Digital computing equipment and devices
Edible fats and oils
Eggs
Eye care goods
Eye care services
Financial services
Fish
Fresh fruit
Fresh or frozen meat (excluding poultry
Fresh or frozen poultry
Fresh vegetables
Furniture
Gasoline
Household appliances
Household textiles
Housekeeping services
Internet access services
Liquor purchased from stores
Liquor served in licensed establishments
Magazines and periodicals
Medicinal and pharmaceutical products
Newspapers
Non-alcoholic beverages
Non-electric kitchen utensils, tableware and cookware
Nuts and seeds
Operation of recreational vehicles
Oral-hygiene produ

In [17]:
len(predictions_dict.keys())

293

In [18]:
with open(test_predictions_path_with_hgru, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)